In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, Input

In [2]:
data= pd.read_csv(r"C:\Users\hsynt\Downloads\Selcuk Abi\Selcuk Abi\VeriSeti2.csv")

In [3]:
# Özellik ve Hedef Ayrımı
X = data.drop(columns=['Target_10', 'adjclose'])
y = data['Target_10']

# Zaman Serisi Formatına Getirme
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length]
        sequences.append(seq)
    return np.array(sequences)

SEQ_LENGTH = 10
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_seq = create_sequences(X_scaled, SEQ_LENGTH)
y_seq = y[SEQ_LENGTH:].values

# Eğitim ve test kümeleri (None durumu için)
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42, stratify=y_seq)

# PCA ve LDA Veri Hazırlığı
X_flat = X_seq.reshape(X_seq.shape[0], -1)  # PCA ve LDA için düzleştirilmiş veri

pca = PCA(n_components=10)
X_pca = pca.fit_transform(X_flat)
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y_seq, test_size=0.2, random_state=42, stratify=y_seq)

lda = LDA(n_components=1)
X_lda = lda.fit_transform(X_flat, y_seq)
X_train_lda, X_test_lda, y_train_lda, y_test_lda = train_test_split(X_lda, y_seq, test_size=0.2, random_state=42, stratify=y_seq)

# Performans Metriklerini Hesaplama
def calculate_metrics(y_true, y_pred, y_pred_proba):
    return {
        "Accuracy": accuracy_score(y_true, y_pred) * 100,
        "Precision": precision_score(y_true, y_pred) * 100,
        "Recall": recall_score(y_true, y_pred) * 100,
        "F1-Score": f1_score(y_true, y_pred) * 100,
        "AUC": roc_auc_score(y_true, y_pred_proba) * 100
    }

# GRU Modeli Tanımlama
def build_gru_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        GRU(64, activation='relu', return_sequences=False),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Performansları Değerlendirme
results_train = []
results_test = []

for reduction, X_tr, X_te, y_tr, y_te in zip(
    ['None', 'PCA', 'LDA'],
    [X_train, X_train_pca, X_train_lda],
    [X_test, X_test_pca, X_test_lda],
    [y_train, y_train_pca, y_train_lda],
    [y_test, y_test_pca, y_test_lda]
):
    if reduction == 'None':
        X_tr = X_tr.reshape(X_tr.shape[0], X_tr.shape[1], X_tr.shape[2])  # 3D giriş
        X_te = X_te.reshape(X_te.shape[0], X_te.shape[1], X_te.shape[2])
    else:
        X_tr = X_tr.reshape(X_tr.shape[0], -1, 1)  # PCA ve LDA için 2D giriş
        X_te = X_te.reshape(X_te.shape[0], -1, 1)

    gru_model = build_gru_model(X_tr.shape[1:])
    gru_model.fit(X_tr, y_tr, epochs=10, batch_size=32, verbose=1)

    # Eğitim Performansı
    y_train_pred = (gru_model.predict(X_tr) > 0.5).astype("int32").flatten()
    y_train_proba = gru_model.predict(X_tr).flatten()
    train_metrics = calculate_metrics(y_tr, y_train_pred, y_train_proba)
    train_metrics['Reduction'] = reduction
    train_metrics['Model'] = 'GRU'
    results_train.append(train_metrics)

    # Test Performansı
    y_test_pred = (gru_model.predict(X_te) > 0.5).astype("int32").flatten()
    y_test_proba = gru_model.predict(X_te).flatten()
    test_metrics = calculate_metrics(y_te, y_test_pred, y_test_proba)
    test_metrics['Reduction'] = reduction
    test_metrics['Model'] = 'GRU'
    results_test.append(test_metrics)

# Eğitim ve Test Sonuçlarını Görselleştirme
train_df = pd.DataFrame(results_train).set_index(["Reduction", "Model"])
test_df = pd.DataFrame(results_test).set_index(["Reduction", "Model"])

# Eğitim Sonuçları Tablosu
print("Eğitim Kümesi Performans Sonuçları:")
print(train_df)

# Test Sonuçları Tablosu
print("\nTest Kümesi Performans Sonuçları:")
print(test_df)

# Sonuçları Kaydetme
train_df.to_csv("gru_pca_lda_train_performance.csv")
test_df.to_csv("gru_pca_lda_test_performance.csv")
print("\nSonuçlar 'gru_pca_lda_train_performance.csv' ve 'gru_pca_lda_test_performance.csv' dosyalarına kaydedildi.")

Epoch 1/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8333 - loss: 0.3948
Epoch 2/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8691 - loss: 0.3119
Epoch 3/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8753 - loss: 0.3002
Epoch 4/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8767 - loss: 0.2925
Epoch 5/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8822 - loss: 0.2855
Epoch 6/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8838 - loss: 0.2786
Epoch 7/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8845 - loss: 0.2762
Epoch 8/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8875 - loss: 0.2714
Epoch 9/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8881 - loss: 0.2707
Epoch 10/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8915 - loss: 0.2632
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1